In [ ]:
%matplotlib inline
import os
import sys
sys.path.append("..")
import time
import numpy as np

from joblib import Parallel, delayed
from skimage.io import imread

from canon.pattern import RescaleExtractor
from canon.util import split_workload

In [ ]:
def process_img(img_files, shape):
    extractor = RescaleExtractor(shape)
    img_data = np.array([imread(f) for f in img_files])
    img_idx = np.array([[int(f[-9:-4])] for f in img_files])
    codes = extractor.features(img_data)
    return np.hstack([img_idx, codes])

def extract_features(jpg_dir, model_name):
    dir_path = os.path.join("img", jpg_dir)
    filenames = [os.path.join(dir_path, filename) for filename in os.listdir(dir_path)]
    print('Found %d files in the directory %s.' % (len(filenames), dir_path))

    fn_grps = split_workload(filenames, 32)

    with Parallel(n_jobs=-1, verbose=12) as parallel:
        data = parallel(delayed(process_img)(grp, model_name) for grp in fn_grps)
        data = np.vstack(data)
        data = data[data[:,0].argsort()]
        print("Loaded a data of shape {}".format(data.shape))
    
    return data

In [ ]:
jpg_dir = "BTO_25C_wb3_60_100"
npy_suffix = "_rescale_16_16"
rescale_shape = (16, 16)
feature_file = jpg_dir + npy_suffix + ".npy"

print("Saving features to " + feature_file)
data = extract_features(jpg_dir, rescale_shape)
np.save("features/" + feature_file, data)